## Databricks Dolly TR Retrieval Embedding

### Import Libraries

In [1]:
import sys
import os

import pandas as pd
import numpy as np
from src.utils import missing_values
import warnings
from datetime import datetime

from datasets import load_dataset

C:\Users\adem\miniconda3\envs\py_env_312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

#### Set Path and Constant Values

In [3]:
# Set Constants

# If use Google Colab
use_colab = False
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT_PATH = "/content/drive/MyDrive/KollektifOgrenme/Hw3-Retrieval/BONUS"  # Google Drive için kök dizin
    sys.path.append(os.path.join(ROOT_PATH, 'src'))
    device = "cuda"
else:
    ROOT_PATH = os.getcwd()
    device = "cpu"

CACHE_PATH = "/content/drive/MyDrive/Colab Notebooks/HW3/cache"
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
EMBEDDING_DATA_PATH = os.path.join(DATA_PATH, 'embedding')

# Create Folders
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(EMBEDDING_DATA_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)

### Load Dataset: Databricks Dolly 15K TR

In [4]:
# https://huggingface.co/datasets/atasoglu/databricks-dolly-15k-tr
# atasoglu/databricks-dolly-15k-tr

ds = load_dataset("atasoglu/databricks-dolly-15k-tr")

In [5]:
df_ins_data_all = ds['train'].to_pandas()
df_ins_data_all.head()

,instruction,context,response,category
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı.",closed_qa
1,Hangisi bir balık türüdür? Tope veya İp,None,top,classification
2,Develer neden susuz uzun süre yaşayabilir?,None,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar.",open_qa
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,Üçüncü kızının adı Alice'dir.,open_qa
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida, 10 Temmuz 1981'de doğdu.",closed_qa


In [6]:
df_ins_data_all["category"].value_counts()

category
open_qa                   3611
general_qa                2191
classification            2136
closed_qa                 1823
brainstorming             1767
information_extraction    1512
summarization             1263
creative_writing           711
Name: count, dtype: int64

In [7]:
df_ins_data_all.columns

Index(['instruction', 'context', 'response', 'category'], dtype='object')

In [8]:
df_ins_data_all.rename(columns={"instruction":"talimat",
                                "context":"giris",
                                "response":"cevap"}, inplace=True)
# df_ins_data_all.drop(labels=["category"])

In [9]:
ds_name = "databricks-dolly-15k-tr"
os.makedirs(os.path.join(RAW_DATA_PATH, ds_name), exist_ok=True)

ds_file_name = os.path.join(RAW_DATA_PATH, ds_name, "turkish_instructions.csv")
df_ins_data_all.to_csv(ds_file_name, index=False, encoding='utf-8')
df_ins_data_all.head()

,talimat,giris,cevap,category
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı.",closed_qa
1,Hangisi bir balık türüdür? Tope veya İp,None,top,classification
2,Develer neden susuz uzun süre yaşayabilir?,None,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar.",open_qa
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,Üçüncü kızının adı Alice'dir.,open_qa
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida, 10 Temmuz 1981'de doğdu.",closed_qa


### Show Metadata, Shape, Statistics and Missing Table

In [10]:
df_ins_data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15014 entries, 0 to 15013
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   talimat   15014 non-null  object
 1   giris     4598 non-null   object
 2   cevap     15014 non-null  object
 3   category  15014 non-null  object
dtypes: object(4)
memory usage: 469.3+ KB


In [11]:
df_ins_data_all.shape

(15014, 4)

In [12]:
missing_values(df=df_ins_data_all, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
giris,10416,69.37525
talimat,0,0.00000
cevap,0,0.00000
category,0,0.00000


In [13]:
col_list = df_ins_data_all.columns.to_list()

for col in col_list:
    df_ins_data_all[f"{col}Length"] = df_ins_data_all[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    df_ins_data_all[f"{col}WordCount"] = df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

In [14]:
df_ins_data_all.head()

,talimat,giris,cevap,category,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,categoryLength,categoryWordCount
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı.",closed_qa,42,6,572,73,99,15,9,1
1,Hangisi bir balık türüdür? Tope veya İp,None,top,classification,39,7,0,0,3,1,14,1
2,Develer neden susuz uzun süre yaşayabilir?,None,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar.",open_qa,42,6,0,0,97,13,7,1
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,Üçüncü kızının adı Alice'dir.,open_qa,69,12,0,0,29,4,7,1
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida, 10 Temmuz 1981'de doğdu.",closed_qa,32,5,1131,163,42,6,9,1


In [15]:
df_ins_data_all[["talimatLength", "girisLength", "cevapLength", "talimatWordCount", "girisWordCount", "cevapWordCount"]].describe(percentiles=[0.80, 0.85, 0.90, 0.95, 0.99]).T

,count,mean,std,min,50%,80%,85%,90%,95%,99%,max
talimatLength,15014.0,73.018316,136.167348,4.0,54.0,96.0,111.0,131.0,169.00,286.00,11914.0
girisLength,15014.0,354.883842,956.829778,0.0,0.0,601.0,796.0,1092.0,1698.70,3975.70,23864.0
cevapLength,15014.0,358.318836,594.308380,1.0,185.0,512.0,622.0,807.0,1198.70,2644.44,26097.0
talimatWordCount,15014.0,9.841415,14.361268,1.0,8.0,13.0,15.0,18.0,22.00,38.00,870.0
girisWordCount,15014.0,46.654389,130.877607,0.0,0.0,79.0,105.0,143.0,220.35,507.61,3796.0
cevapWordCount,15014.0,48.086319,80.956700,1.0,25.0,68.0,83.0,107.0,158.35,351.00,4274.0


In [16]:
# df_ins_data_all[df_ins_data_all["talimatWordCount"]==1]

In [17]:
# df_ins_data_all[df_ins_data_all["talimatWordCount"]==870]

### Split Data

In [18]:
df_ins_data_all['soru'] = df_ins_data_all['talimat'].fillna('') + ' ' + df_ins_data_all['giris'].fillna('')
df_ins_data_all[["talimat", "giris", "soru", "cevap"]].head()

,talimat,giris,soru,cevap
0,Virgin Australia ne zaman faaliyete geçti?,"Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia ne zaman faaliyete geçti? Virgin Australia Airlines Pty Ltd'nin ticari adı olan Virgin Australia, Avustralya merkezli bir havayolu şirketidir. Filo büyüklüğüne göre Virgin markasını kullanan en büyük havayolu şirketidir. 31 Ağustos 2000'de Virgin Blue adıyla, tek hat üzerinde iki uçakla hizmete başlamıştır.[3] Eylül 2001'de Ansett Australia'nın iflas etmesinden sonra birdenbire kendisini Avustralya iç pazarında önemli bir havayolu şirketi olarak buldu. Havayolu o zamandan beri Brisbane, Melbourne ve Sidney'deki merkezlerden Avustralya'nın 32 şehrine doğrudan hizmet verecek şekilde büyüdü.[4]","Virgin Australia, 31 Ağustos 2000'de Virgin Blue olarak, tek bir rotada iki uçakla hizmete başladı."
1,Hangisi bir balık türüdür? Tope veya İp,None,Hangisi bir balık türüdür? Tope veya İp,top
2,Develer neden susuz uzun süre yaşayabilir?,None,Develer neden susuz uzun süre yaşayabilir?,"Develer, hörgüçlerini uzun süre enerji ve su ile doldurmak için hörgüçlerindeki yağı kullanırlar."
3,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",None,"Alice'in ailesinin üç kızı var: Amy, Jessy ve üçüncü kızın adı nedir?",Üçüncü kızının adı Alice'dir.
4,Tomoaki Komorida ne zaman doğdu?,"Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz 2010'da Japonya'ya döndü ve J2 kulübü Giravanz Kitakyushu'ya katıldı. Emekli olduğu 2012 yılına kadar sık ​​sık defansif orta saha oyuncusu ve stoper olarak oynadı.","Tomoaki Komorida ne zaman doğdu? Komorida, 10 Temmuz 1981'de Kumamoto Eyaletinde doğdu. Liseden mezun olduktan sonra 2000 yılında J1 League kulübü Avispa Fukuoka'ya katıldı. 2001'de orta saha oyuncusu olarak çıkış yapmasına rağmen fazla oynamadı ve kulüp 1. 2001 sezonunun sonunda J2 Ligi. 2002'de J2 kulübü Oita Trinita'ya taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu oldu ve kulüp 2002'de şampiyonluğu kazandı ve 2003'te terfi etti. 2005'e kadar birçok maç oynadı. Eylül 2005'te J2 kulübü Montedio Yamagata'ya geçti. 2006'da J2 kulübü Vissel Kobe'ye taşındı. Defansif orta saha oyuncusu olarak normal bir oyuncu olmasına rağmen, yaz aylarında giderek daha az oynandı. 2007'de, yerel bölgesinde bulunan Japonya Futbol Ligi kulübü Rosso Kumamoto'ya (daha sonra Roasso Kumamoto) taşındı. Normal bir oyuncu olarak oynadı ve kulüp 2008'de J2'ye yükseldi. Eskisi kadar oynamasa da yine de birçok maçta forma giydi. 2010 yılında Endonezya'ya taşındı ve Persela Lamongan'a katıldı. Temmuz

In [19]:
df_sample_1 = df_ins_data_all[df_ins_data_all['giris'].notna()].sample(n=1000, random_state=42)
df_sample_1_idx = df_sample_1.index
df_sample_1.head()

df_sample_2 = df_ins_data_all[~(df_ins_data_all['giris'].notna())].sample(n=1000, random_state=42)
df_sample_2_idx = df_sample_2.index
df_sample_2.head()

df_sample = pd.concat([df_sample_1, df_sample_2])
df_sample['soru'] = df_sample['talimat'].fillna('') + " " + df_sample['giris'].fillna('')

df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

sampling_file_name = os.path.join(RAW_DATA_PATH, ds_name, "sampled_turkish_instructions.csv")
df_sample.to_csv(sampling_file_name, index=False)

df_sample = df_sample[["talimat", "giris", "soru", "cevap"]].reset_index(drop=True)

df_sample = pd.read_csv(sampling_file_name)
df_sample.head()

talimat  \
0                                                                                           Kara ceviz ağaçları nerede yetişir?   
1                                                                     Gunshy albümünü yayınladığında Josh Logan kaç yaşındaydı?   
2  Aşağıdaki paragrafa göre, TENS (transkutanöz elektriksel sinir stimülasyonu) tedavisinin bazı potansiyel faydaları nelerdir?   
3                                                                                                            Baur au Lac nedir?   
4                                                                                                Nero ne zaman ve nerede doğdu?   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

In [20]:
df_sample.sample(5)

,talimat,giris,cevap,category,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,categoryLength,categoryWordCount,soru
213,Everest Dağı nerede bulunur?,"Everest Dağı, Himalayaların Mahalangur Himal alt aralığında bulunan, Dünya'nın deniz seviyesinden en yüksek dağıdır. Çin-Nepal sınırı, zirve noktasından geçiyor. 8.848,86 m rakımı en son 2020'de Çinli ve Nepalli yetkililer tarafından belirlendi.","Everest Dağı, Himalayaların Mahalangur Himal alt bölgesinde yer almaktadır. Zirve noktalarından geçen Çin ve Nepal ile sınır komşusudur.",information_extraction,28,4,245,31,136,18,22,1,"Everest Dağı nerede bulunur? Everest Dağı, Himalayaların Mahalangur Himal alt aralığında bulunan, Dünya'nın deniz seviyesinden en yüksek dağıdır. Çin-Nepal sınırı, zirve noktasından geçiyor. 8.848,86 m rakımı en son 2020'de Çinli ve Nepalli yetkililer tarafından belirlendi."
1452,Neden iki takım diş çıkarıyoruz?,NaN,"Tam bir diş setini değiştirmek garip gelebilir, ancak ilk süt dişleriniz siz beş veya altı yaşlarına geldiğinizde görevini yapmış olur. Süt dişleri, vücudunuzun daha kapsamlı bir yetişkin diş setini barındırma zamanı geldiğinde, temel besinleri çiğnemenize ve konuşmayı öğrenmenize yardımcı olmuştur. Süt dişlerinizi kaybedersiniz ve yetişkin dişleriniz olur.",open_qa,32,5,0,0,359,47,7,1,Neden iki takım diş çıkarıyoruz?
1006,İntikamcılar kimlerdir?,NaN,"Avengers, Marvel çizgi romanlarındaki süper kahramanlardan oluşan bir ekiptir. Harikalar'da birçok süper kahraman var ve bunlardan birkaçına intikamcı deniyor. Ekip, süper kahramanlardan oluşuyor: Demir Adam, Kaptan Amerika, Şahin-göz, Wanda, Vision, Kaptan Marvel, Hulk, Thor, Karınca Adam, Kara Panter, Kış Askeri, Kara Dul ve diğerleri.",creative_writing,23,2,0,0,339,44,16,1,İntikamcılar kimlerdir?
1855,Gelir ve emlak vergileri arasındaki fark nedir?,NaN,"Gelir vergileri, bireyin çalışarak kazandığı para üzerinden alınan vergilerdir. Emlak vergileri, bir bireyin sahip olduğu mülkün değeri üzerinden alınan vergilerdir.",open_qa,47,7,0,0,165,20,7,1,Gelir ve emlak vergileri arasındaki fark nedir?
626,Gary Collins NHL'de oynadı mı?,"Ranleigh Gary Collins (27 Eylül 1935 - 17 Haziran 2022), 1958–59 sezonunda Toronto Maple Leafs için Ulusal Hokey Ligi'nde iki playoff maçı oynayan Kanadalı bir buz hokeyi oyuncusuydu. 1956'dan 1968'e kadar süren kariyerinin geri kalanı ikinci liglerde geçti.\n\nCollins, Haziran 2022'de 86 yaşında öldü.[1]","Ranleigh Gary Collins (27 Eylül 1935 - 17 Haziran 2022), 1958–59 sezonunda Toronto Maple Leafs için Ulusal Hokey Ligi'nde iki playoff maçı oynayan Kanadalı bir buz hokeyi oyuncusuydu. 1956'dan 1968'e kadar süren kariyerinin geri kalanı ikinci liglerde geçti.\n\nCollins, Haziran 2022'de 86 yaşında öldü.",summarization,30,5,304,44,301,44,13,1,"Gary Collins NHL'de oynadı mı? Ranleigh Gary Collins (27 Eylül 1935 - 17 Haziran 2022), 1958–59 sezonunda Toronto Maple Leafs için Ulusal Hokey Ligi'nde iki playoff maçı oynayan Kanadalı bir buz hokeyi oyuncusuydu. 1956'dan 1968'e kadar süren kariyerinin geri kalanı ikinci liglerde geçti.\n\nCollins, Haziran 2022'de 86 yaşında öldü.[1]"


In [21]:
df_sample.shape

(2000, 13)

### Embedding Models

#### Description

| **No** | **Model Adı**                           | **Parametre Sayısı** | **Boyut Sayısı (Dimension Size)** | **Açıklama (Description)**                                                                                     | **Özel Amaçlı Eğitim**                   | **Eğitim Veri Kümesi**                                                        | **Desteklenen Diller**        |
|--------|----------------------------------------|----------------------|----------------------------------|---------------------------------------------------------------------------------------------------------------|-----------------------------------------|-----------------------------------------------------------------------------|---------------------------------|
| 1      | **sentence-transformers/all-MiniLM-L12-v2** | 33.4M                | 384                              | Cümleleri ve kısa paragrafları 384 boyutlu yoğun bir vektör uzayına dönüştüren bir modeldir. Bu model, cümle benzerliği, kümeleme ve anlamsal arama gibi görevlerde kullanılabilir. ([HuggingFace](https://huggingface.co/sentence-transformers/all-MiniLM-L12-v2)) | Microsoft/MiniLM-L12-H384-uncased modeli üzerinde ince ayar yapılmıştır. | Cümle çiftleri (NLI ve Paraphrase Corpus)                                     | İngilizce                       |
| 2      | **jinaai/jina-embeddings-v3**           | 572M                 | 1536                             | Jina AI tarafından geliştirilen bu model, metinleri 1536 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/jinaai/jina-embeddings-v3))         | -                                       | Özel büyük ölçekli çok dilli veri kümesi                                      | Çok Dilli (50+ dil)            |
| 3      | **intfloat/multilingual-e5-large-instruct** | 560M                 | 1024                             | Çok dilli metinleri 1024 boyutlu vektörlere dönüştürebilen bir modeldir. ([HuggingFace](https://huggingface.co/intfloat/multilingual-e5-large-instruct))                                | -                                       | mC4, CCNet, MassiveText gibi büyük veri kümeleri                              | Çok Dilli (100+ dil)           |
| 4      | **BAAI/bge-m3**                         | -                    | 1024                             | Beijing Academy of Artificial Intelligence (BAAI) tarafından geliştirilen bu model, metinleri 1024 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/BAAI/bge-m3)) | -                                       | OpenAI GPT benzeri veri kümesi (web crawl)                                    | İngilizce, Çince               |
| 5      | **ytu-ce-cosmos/turkish-colbert**       | 111M                 | 768                              | Yıldız Teknik Üniversitesi tarafından geliştirilen bu model, Türkçe metinleri 128 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/ytu-ce-cosmos/turkish-colbert)) | Türkçe metinler üzerinde özel olarak eğitilmiştir. | Türkçe metin veri kümeleri                                                  | Türkçe                         |
| 6      | **thenlper/gte-large**                  | 335M                 | 768                              | The NLP ER tarafından geliştirilen bu model, metinleri 768 boyutlu vektörlere dönüştürmek için tasarlanmıştır. ([HuggingFace](https://huggingface.co/thenlper/gte-large))             | -                                       | Çoklu cümle çiftleri ve NLI veri kümeleri                                     | İngilizce                       |


In [27]:
from sentence_transformers import SentenceTransformer

embedding_models = [
    # 'sentence-transformers/all-MiniLM-L12-v2',
    # "jinaai/jina-embeddings-v3",
    # "intfloat/multilingual-e5-large-instruct",
    # "BAAI/bge-m3",
    "ytu-ce-cosmos/turkish-colbert",
    # "thenlper/gte-large"
]

In [28]:
def load_or_download_model(model_name, device):
    model_path = os.path.join(CACHE_PATH, model_name)
    if os.path.exists(model_path):
        print(f"Loading Cached Model: {model_name}")
        return SentenceTransformer(model_path, device=device, trust_remote_code=True)
    else:
        print(f"Downloading Model: {model_name}")
        model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
        model.save(model_path)
        return model

In [29]:
ds_name = "databricks-dolly-15k-tr"
os.makedirs(os.path.join(EMBEDDING_DATA_PATH, ds_name), exist_ok=True)

In [31]:
for idx, model_name in enumerate(embedding_models):
    try:
        embedding_start_time = datetime.now()
        print(f"{idx+1}-Embedding Model Name: {model_name} - {embedding_start_time}")
        model = load_or_download_model(model_name, device)

        # Sorular için embedding
        print("Sorular için embedding hesaplanıyor...")
        if model_name == "ytu-ce-cosmos/turkish-colbert":
            # df_sample['soru'] = df_sample['soru'].str.lower()
            df_sample['soru'] = df_sample['soru'].apply(lambda text: text.replace("I", "ı").lower())
        question_embeddings = model.encode(df_sample['soru'].tolist(), show_progress_bar=True)
        question_embedding_dim = len(question_embeddings[0])
        print(f"Embedding Dimension: {question_embedding_dim}")
        question_columns = [f'question_embedding_{i}' for i in range(question_embedding_dim)]
        question_df = pd.DataFrame(question_embeddings, columns=question_columns)
        question_df = pd.concat([df_sample[['soru']], question_df], axis=1)

        question_output_file = os.path.join(
          EMBEDDING_DATA_PATH, ds_name, f"question_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        question_df.to_csv(question_output_file, compression='gzip', index=False)

        # Cevaplar için embedding
        print("Cevaplar için embedding hesaplanıyor...")
        if model_name == "ytu-ce-cosmos/turkish-colbert":
            # df_sample['cevap'] = df_sample['cevap'].str.lower()
            df_sample['cevap'] = df_sample['cevap'].apply(lambda text: text.replace("I", "ı").lower())
        answer_embeddings = model.encode(df_sample['cevap'].tolist(), show_progress_bar=True)
        answer_embedding_dim = len(answer_embeddings[0])
        print(f"Embedding Dimension: {answer_embedding_dim}")
        answer_columns = [f'answer_embedding_{i}' for i in range(answer_embedding_dim)]
        answer_df = pd.DataFrame(answer_embeddings, columns=answer_columns)
        answer_df = pd.concat([df_sample[['cevap']], answer_df], axis=1)
        answer_output_file = os.path.join(
          EMBEDDING_DATA_PATH, ds_name, f"answer_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        answer_df.to_csv(answer_output_file, compression='gzip', index=False)

        print(f"{model_name} için embedding işlemi tamamlandı. Embedding Duration: {datetime.now() - embedding_start_time}")
        print("+ " * 15)
    except Exception as exc:
        print(f"{model_name} için hata oluştu: {exc}")
        print("x " * 15)

No sentence-transformers model found with name ytu-ce-cosmos/turkish-colbert. Creating a new one with mean pooling.


1-Embedding Model Name: ytu-ce-cosmos/turkish-colbert - 2024-12-25 14:02:32.563595
Sorular için embedding hesaplanıyor...


Batches: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [5:47:42<00:00, 331.15s/it]


Embedding Dimension: 768
Cevaplar için embedding hesaplanıyor...


Batches: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 63/63 [04:18<00:00,  4.11s/it]


Embedding Dimension: 768
ytu-ce-cosmos/turkish-colbert için embedding işlemi tamamlandı. Embedding Duration: 5:53:40.833307
+ + + + + + + + + + + + + + + 


#### Embedding Vectors

In [32]:
for idx, f in enumerate(os.listdir(EMBEDDING_DATA_PATH + "/" + ds_name)):
    # print(f)
    # if f.split("_")[0] == "answer":
    if f.split("_")[0] in ["answer", "question"]:
        df_emb_data = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH + "/" + ds_name, f), compression='gzip')
        print(f"{idx+1} - Emb. File Name:{f} & Shape: {df_emb_data.shape}")
        print(df_emb_data.head(0))
        # print(f"Columns:{df_emb_data.columns.to_list()}")
        print("+ " * 25)

1 - Emb. File Name:answer_BAAI__bge-m3_embeddings.csv.gz & Shape: (2000, 1025)
Empty DataFrame
Columns: [cevap, answer_embedding_0, answer_embedding_1, answer_embedding_2, answer_embedding_3, answer_embedding_4, answer_embedding_5, answer_embedding_6, answer_embedding_7, answer_embedding_8, answer_embedding_9, answer_embedding_10, answer_embedding_11, answer_embedding_12, answer_embedding_13, answer_embedding_14, answer_embedding_15, answer_embedding_16, answer_embedding_17, answer_embedding_18, answer_embedding_19, answer_embedding_20, answer_embedding_21, answer_embedding_22, answer_embedding_23, answer_embedding_24, answer_embedding_25, answer_embedding_26, answer_embedding_27, answer_embedding_28, answer_embedding_29, answer_embedding_30, answer_embedding_31, answer_embedding_32, answer_embedding_33, answer_embedding_34, answer_embedding_35, answer_embedding_36, answer_embedding_37, answer_embedding_38, answer_embedding_39, answer_embedding_40, answer_embedding_41, answer_embedding